In [ ]:
import math
from collections import Counter
from itertools import combinations

import attr
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
px.defaults.height = 600
from scipy.stats import fisk
from scipy.integrate import quad as integrate


rng = np.random.default_rng()


@attr.s
class ProbWindowSize:
    w = attr.ib()

    def _delta_idxs_to_mask(self, idxs):
        mask = np.zeros_like(self.w, bool)
        mask[np.array(idxs, dtype=int)] = True
        return mask

    def p_event(self, delta_idxs):
        w    = self.w
        mask = self._delta_idxs_to_mask(delta_idxs)
        p    = w[~mask].prod() * (1 - w[mask]).prod()
        assert p >= 0

        return p

    def pd_delta_k(self, k):
        return sum(map(
            self.p_event,
            combinations(range(len(self.w)), k)
        ))

    def cd_delta_k(self, k):
        return sum(map(self.pd_delta_k, range(k+1)))

def plot_windows(ws, x=None):
    if x is None:
        x = np.arange(0, max(map(len, ws.values())))

    pdfs = plt.subplot(211)
    cdfs = plt.subplot(212)
    pdfs.set_xlabel('$k$')
    cdfs.set_xlabel('$k$')
    pdfs.set_ylabel('$P(\delta(w)=k)$')
    cdfs.set_ylabel('$P(\delta(w)\leq k)$')

    for label, w in ws.items():
        y = np.vectorize(ProbWindowSize(w).pd_delta_k)(x)
        assert len(y[y<0]) == 0, y
        pdfs.plot(x, y, label=label)
        cdfs.plot(x, np.cumsum(y), label=label)

    plt.legend()

# Communication Model

## Model

Three nodes:

 * $A,B$ - receivers
 * $I$ - transaction issuer

When $I$ creates a transaction, she sends it to both $A$ and $B$.
After $t_A,t_B$ time, it recieved by $A,B$ respectively.
$t_A,t_B \sim LL$ (Log-logistic)

The probability that a transaction that was received $t_A$ seconds ago in $A$, is included in $B$, is given by:
$$
P(tx \in B | t_A) = \int_{0}^{\infty} f(t)\cdot F(t + t_A) \,dt
$$
(the 'now' is the sync point)

Where $f,F$ are the probability density and cumulative distribution functions of $LL$, respectively:
$$
f(t) = \frac{\beta t^{\beta-1}}{(1+t^\beta)^2}
\;\;\;\;\;
F(t) = \frac{1}{1+t^{-\beta}}
$$

The following code numerically integrates $f,F$ to get the prob.

In [ ]:
class PrTxInB:
    def __init__(self, p = fisk(c=2)):
        self.p = p
        self.pr_for_t = np.vectorize(self._pr_for_t)

    def _gen_inner_conv(self, age):
        return lambda t: self.p.pdf(t) * self.p.cdf(t + age)

    def _pr_for_t(self, age):
        return integrate(self._gen_inner_conv(age), 0, np.inf)[0]

x = np.linspace(0, 10, 100)
p = PrTxInB()

$LL$ PDF:

In [ ]:
plt.plot(x, p.p.pdf(x))
plt.xlabel('$t$')
plt.ylabel('$f(t)$')
plt.show()

In [ ]:
plt.plot(x, p.pr_for_t(x))
plt.xlabel('$t_A$')
plt.ylabel('$P(tx \in B | t_A)$')
plt.show()

# $P(tx \in B) \Longrightarrow P(\delta(w)=k)$

In [ ]:
ws = { f'U{i}': np.repeat(.1, 10) * i for i in [1, 3, 5, 7, 9] }

plot_windows(ws, np.arange(0, 12))

In [ ]:
ws = {
    'C_1_9': np.array([.1] * 1 + [.9] * 9),
    'C_3_7': np.array([.1] * 3 + [.9] * 7),
    'C_5_5': np.array([.1] * 5 + [.9] * 5),
    'C_7_3': np.array([.1] * 7 + [.9] * 3),
    'C_9_1': np.array([.1] * 9 + [.9] * 1),
}

plot_windows(ws)

In [ ]:
ws = {
    'lin': np.linspace (.1, .9, 13),
    'log': np.geomspace(.1, .9, 13),
}

plot_windows(ws)

## Interactive exploration

In [ ]:
ns = [
    widgets.IntSlider(
        description=f'#p={i/10}',
        value=3,
        min=0,
        max=10,
        continuous_update=False,
        orientation='vertical',
        # readout_format='.1f',
    )
    for i in range(0, 11, 2)
]

In [ ]:
min_max_step = 0.0, 1.0, 0.1
@widgets.interact
def f(
    n000=ns[0],
    n020=ns[1],
    n040=ns[2],
    n060=ns[3],
    n080=ns[4],
    n100=ns[5],
):
    plot_windows(
        {'-': np.array(
            [0.0] * n000 +
            [0.2] * n020 +
            [0.4] * n040 +
            [0.6] * n060 +
            [0.8] * n080 +
            [1.0] * n100
        )},
        np.arange(0, 1+ n000 + n020 + n040 + n060 + n080 + n100)
    )

widgets.HBox(ns)

# Reconciliation using fixed-size protocols

In [ ]:
def split_probs(split):
    return [ProbWindowSize(w).cd_delta_k(delta_threshold) for w in split]

In [ ]:
delta_threshold=2
w = np.linspace(.1, .9, 9)
# w = np.array([.1]*6 + [.9]*6)
w

In [ ]:
wise_split = [2, 2]
n_bins = 1+len(wise_split)
wise_p = np.split(w, np.cumsum(wise_split))
wise_p

In [ ]:
sp = split_probs(wise_p)
print(np.prod(sp))
print(sp)

In [ ]:
wise_p = [
    np.array([.1, .6, .8]),
    np.array([.2, .4, .9]),
    np.array([.3, .5, .7])
]

In [ ]:
sp = split_probs(wise_p)
print(np.prod(sp))
print(sp)

In [ ]:
res = []
for i in range(10000):
    sw = np.split(rng.permutation(w), n_bins)
    sp = split_probs(sw)
    res.append((np.prod(sp), sp, sw))

In [ ]:
prods = np.array([r[0] for r in res])

In [ ]:
res[prods.argmin()]

In [ ]:
res[prods.argmax()]

In [ ]:
prods.mean()

## Experiment:

Assumptions:
- $B \subseteq A$
- Set reconciliaction protocol is given with $M$ bits.

Comparisons:
- 1 instance with $t*M$ memory
- $t$ instances, random split sets
- $t$ instances, smart allocation by probability

Start with CPI as it has easy-to-compute properties:
$1/0$ success depends on the predefined threshold, and number of bits given threshold.

In [ ]:
P = np.array([
    [10, 1000],
    [90, 1000],
])

expected_delta = math.ceil(P.prod(1).sum() / 100)
expected_delta

In [ ]:
t = 4

In [ ]:
class CPI:
    @staticmethod
    def calc_m(d, b=32):
        return (d+1)*(b+1)-1

    @staticmethod
    def calc_d(m, b=32):
        return (m+1)//(b+1)-1

    @staticmethod
    def estim_c(d, b=32):
        return b + d**3


trials = 100000
def empirical_delta(P, trials=trials):
    return sum(rng.binomial(p[1], p[0]/100, trials) for p in P)

M = CPI.calc_m(d=expected_delta)
M

### Set $B$ defined empirically by probability

#### Single instance

In [ ]:
(empirical_delta(P) <= expected_delta).sum() / trials

#### $t$ instances

In [ ]:
max_delta = CPI.calc_d(M//t)
max_delta

In [ ]:
def random_split(a, t):
    return np.split(rng.permutation(a), t)

sum(
    (np.array([empirical_delta(Counter(a).items(), trials=1) for a in random_split(np.repeat(P.T[0], P.T[1]), t)]).ravel() <= max_delta).all()
    for _ in range(trials)
) / trials

#### $t$ instances with $p$-aware allocation

In [ ]:
sum(
    (np.array([empirical_delta(Counter(a).items(), trials=1) for a in [Counter({10:25, 90:25}) for _ in range(t)]]).ravel() <= max_delta).all()
    for _ in range(trials)
) / trials

### Set $B$ is predefined

In [ ]:
B = P.prod(1)//100
B

In [ ]:
a = np.repeat(P.T[0], P.T[1])
a = np.vstack([a, np.zeros_like(a)])

idx = 0
for (p,c),b in zip(P,B):
    a[1, idx:idx+b] = 1
    idx+=c

#### Single instance

In [ ]:
B.sum() <= CPI.calc_d(M)

#### $t$ instances

In [ ]:
Mp = int(M*1.06)
Mp

In [ ]:
max_delta = CPI.calc_d(Mp//t)
max_delta

In [ ]:
sum(
    (np.stack(random_split(a[1], t)).sum(1) <= max_delta).all()
    for _ in range(trials)
) / trials

#### $t$ instances with $p$-aware allocation

In [ ]:
def split_by_priors(a):
    return np.split(a, np.cumsum(P[:-1,1]), axis=1)

In [ ]:
sum(
    (np.stack([np.stack(random_split(part[1], t)) for part in split_by_priors(a)]).sum(2).sum(0) <= max_delta).all()
    for _ in range(trials)
) / trials

# Directly using $P(tx \in B)$

## Simplest setting
One shot sending, if failed send the whole set.

- $n$ - whole set size
- $m$ - first send size
- $d$ - the difference size $\delta$
- $P(F|m,d)$ - probability to fail based on $m,d$